In [1]:
#https://openweathermap.org/current#geo
#https://openweathermap.org/
appid = "0b0ee9d705804b13cc0e0b4f71d6912b"# полученный при регистрации на OpenWeatherMap.org. Что-то вроде такого набора букв и цифр: "6d8e495ca73d5bbc1d6bf8ebd52c4123"

import pandas as pd
import time
import requests
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

timestamp = int(time.time()-24*60*60)
df_ = pd.DataFrame()
df_onecall = pd.DataFrame()
df_forecast = pd.DataFrame()
#city_id = ['ГТП-2','ГТП-4','ГТП-7','ГТП-12','ГТП-13','ГТП-16','ГТП-NK','ГТП-20','ГТП-22','ГТП-46']
#city = ['Егозово','Киселевск','Черногорск','назарово','канск','ванино','мухоршибирь','чегдомын','артем','Мурманск']
#city_id = ['1500665','1503277','1508054','1497951','1504682','2119932','2019710','2025579','524305']
city = [['54.711319135715385','86.10950437082472'],['54.00639388698265','86.65903268718424'],['53.823218423466514','91.27580958207747'],['56.031420657404055','90.35205564086384'],
       ['56.19584710479182','95.67773888241815'],['49.09796792354582','140.25748941094645'],['51.04499777536743','107.81285875286638'],['51.123249338172016','132.99207097836057'],
       ['43.35543561671812','132.17076196895238'],['68.9792', '33.0925']]

# Исторические данные
for  k in city:
    df = pd.DataFrame()
    try:
        # Исторические данные
        df = pd.DataFrame()

        res = requests.get("https://api.openweathermap.org/data/2.5/onecall/timemachine",
                           params={'lat': k[0], 'lon': k[1], 'dt':timestamp, 'APPID': appid, 'units': 'metric'})
        data = res.json()
        
        for i in data['hourly']:
            df = df.append({'1d': datetime.utcfromtimestamp(i['dt']).strftime('%Y-%m-%d'), 
                            '2T': i['temp'], 
                            '3H': i['pressure']*0.75,
                            '4fi': i['humidity'],
                            '5V': i['wind_speed']}, ignore_index=True)
        df['1d']= pd.to_datetime(df['1d'])
        df = df.groupby('1d').mean()
        
    except Exception as e:
        print("Exception (forecast):", e)
        pass
    
    if len(df_onecall)==0:
        
        df_onecall=df.copy()
    else:
        df_onecall=pd.merge(df_onecall, df, how ='inner', on ='1d')
        
df_onecall

,2T_x,3H_x,4fi_x,5V_x,2T_y,3H_y,4fi_y,5V_y,2T_x,3H_x,...,4fi_y,5V_y,2T_x,3H_x,4fi_x,5V_x,2T_y,3H_y,4fi_y,5V_y
1d,,,,,,,,,,,,,,,,,,,,,
2022-01-22,-21.01625,776.21875,98.5,0.75375,-19.293333,775.78125,97.291667,0.7825,-23.38,774.46875,...,97.25,0.79125,-13.385,770.78125,50.625,4.666667,-15.859167,760.78125,90.958333,5.035


In [2]:
#прогноз на 5 дней
for  k in city:
    df = pd.DataFrame()
    try:
        res = requests.get("http://api.openweathermap.org/data/2.5/forecast",
                           params={'lat': k[0], 'lon': k[1], 'units': 'metric', 'lang': 'ru', 'APPID': appid})
        data = res.json()
        #print(data)
        print( data['city']['name'])
        
        for i in data['list']:
            df = df.append({'1d': (i['dt_txt'])[:10], 
                            '2T': i['main']['temp'], 
                            '3H': i['main']['pressure']*0.75,
                            '4fi': i['main']['humidity'],
                            '5V': i['wind']['speed']}, ignore_index=True)
        df['1d']= pd.to_datetime(df['1d'])
        df = df.groupby('1d').mean()
    except Exception as e:
        print("Exception (forecast):", e)
        pass
    
    if len(df_forecast)==0:
        df_forecast=df.copy()
    else:
        df_forecast=pd.merge(df_forecast, df, how ='inner', on ='1d')

df_forecast

Ленинск-Кузнецкий
Киселёвск
Черногорск
Назарово
Канск
Ванино
Мухоршибирь
Чегдомын
Артём
Мурманск


,2T_x,3H_x,4fi_x,5V_x,2T_y,3H_y,4fi_y,5V_y,2T_x,3H_x,...,4fi_y,5V_y,2T_x,3H_x,4fi_x,5V_x,2T_y,3H_y,4fi_y,5V_y
1d,,,,,,,,,,,,,,,,,,,,,
2022-01-23,-16.564000,776.85000,96.400000,1.092000,-15.43600,776.55000,96.000000,0.78200,-22.066000,777.30000,...,97.600000,0.92400,-14.00800,770.25000,79.000,2.664000,0.656000,749.55000,87.800000,7.17000
2022-01-24,-21.870000,780.18750,98.875000,1.516250,-19.93250,780.00000,98.250000,0.96000,-21.362500,780.56250,...,97.000000,0.77750,-12.31875,769.03125,81.125,3.085000,-0.685000,744.46875,86.375000,7.74125
2022-01-25,-20.548750,777.84375,97.000000,1.812500,-20.51250,778.12500,97.625000,1.19000,-22.791250,779.43750,...,97.250000,0.76500,-11.77875,765.93750,82.875,2.381250,-6.040000,751.78125,84.375000,8.09500
2022-01-26,-16.900000,775.12500,95.000000,1.862500,-15.28375,775.21875,94.500000,1.58125,-23.530000,778.68750,...,97.375000,0.75750,-11.43500,767.90625,83.375,3.221250,-7.450000,751.12500,91.375000,4.09125
2022-01-27,-17.621250,777.28125,96.500000,2.038750,-15.55500,777.56250,93.875000,1.77375,-16.407500,778.03125,...,97.250000,0.62125,-11.98125,769.03125,80.625,2.517500,-7.745000,751.50000,97.250000,1.65250
2022-01-28,-22.366667,779.50000,98.333333,1.136667,-19.97000,779.00000,97.333333,0.71000,-12.786667,778.25000,...,89.333333,0.99000,-8.47000,765.00000,63.000,1.763333,-14.646667,754.50000,99.666667,1.36000


In [3]:
df_=pd.concat([df_onecall,df_forecast])        
pos_= df_.index.values[0]
cols = [0]
metadata = pd.read_excel('/home/playground/Potreb/Data/Weather.xlsx',usecols=cols, engine='openpyxl')
metadata = metadata.iloc[1:]
metadata['1d']= pd.to_datetime(metadata['Unnamed: 0'])
metadata_ = metadata[(metadata['1d']==pos_)]
pos=metadata_.index[0]
df_.tail(8)

,2T_x,3H_x,4fi_x,5V_x,2T_y,3H_y,4fi_y,5V_y,2T_x,3H_x,...,4fi_y,5V_y,2T_x,3H_x,4fi_x,5V_x,2T_y,3H_y,4fi_y,5V_y
1d,,,,,,,,,,,,,,,,,,,,,
2022-01-22,-21.016250,776.21875,98.500000,0.753750,-19.293333,775.78125,97.291667,0.78250,-23.380000,774.46875,...,97.250000,0.79125,-13.38500,770.78125,50.625,4.666667,-15.859167,760.78125,90.958333,5.03500
2022-01-23,-16.564000,776.85000,96.400000,1.092000,-15.436000,776.55000,96.000000,0.78200,-22.066000,777.30000,...,97.600000,0.92400,-14.00800,770.25000,79.000,2.664000,0.656000,749.55000,87.800000,7.17000
2022-01-24,-21.870000,780.18750,98.875000,1.516250,-19.932500,780.00000,98.250000,0.96000,-21.362500,780.56250,...,97.000000,0.77750,-12.31875,769.03125,81.125,3.085000,-0.685000,744.46875,86.375000,7.74125
2022-01-25,-20.548750,777.84375,97.000000,1.812500,-20.512500,778.12500,97.625000,1.19000,-22.791250,779.43750,...,97.250000,0.76500,-11.77875,765.93750,82.875,2.381250,-6.040000,751.78125,84.375000,8.09500
2022-01-26,-16.900000,775.12500,95.000000,1.862500,-15.283750,775.21875,94.500000,1.58125,-23.530000,778.68750,...,97.375000,0.75750,-11.43500,767.90625,83.375,3.221250,-7.450000,751.12500,91.375000,4.09125
2022-01-27,-17.621250,777.28125,96.500000,2.038750,-15.555000,777.56250,93.875000,1.77375,-16.407500,778.03125,...,97.250000,0.62125,-11.98125,769.03125,80.625,2.517500,-7.745000,751.50000,97.250000,1.65250
2022-01-28,-22.366667,779.50000,98.333333,1.136667,-19.970000,779.00000,97.333333,0.71000,-12.786667,778.25000,...,89.333333,0.99000,-8.47000,765.00000,63.000,1.763333,-14.646667,754.50000,99.666667,1.36000


In [4]:

def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
   
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()

append_df_to_excel('/home/playground/Potreb/Data/Weather.xlsx', df_, sheet_name='His', index=None, startrow=pos+1,startcol = 1, header=None)   
